## Fall Detection using Visual Tracking
### Computer Vision and Image Analysis
-------
##### Katarina Keishanti Joanne Kartakusuma - 21/472847/PA/20347
#### Assignment 3

### Import Dependencies

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

### Drawing keypoints

In [ ]:
def draw_kp(frame, keypoints, confidence_thresh):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_thresh:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0, 255, 0), -1)

### Connecting the keypoints

In [ ]:
EDGES = {
    (0, 1): "m",
    (0, 2): "c",
    (1, 3): "m",
    (2, 4): "c",
    (0, 5): "m",
    (0, 6): "c",
    (5, 7): "m",
    (7, 9): "m",
    (6, 8): "c",
    (8, 10): "c",
    (5, 6): "y",
    (5, 11): "m",
    (6, 12): "c",
    (11, 12): "y",
    (11, 13): "m",
    (13, 15): "m",
    (12, 14): "c",
    (14, 16): "c",
}

In [ ]:
def kp_connections(frame, keypoints, edges, confidence_thresh):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]

        if (c1 > confidence_thresh) & (c2 > confidence_thresh):
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)

### Fall detection

In [ ]:
def detect_falls(video_path):
    interpreter = tf.lite.Interpreter(model_path="/Users/katarinajoanne/University/UGM/2 - Second Year/Fourth Semester/Computer Vision and Image Analysis/Assignment-3/lite-model_movenet_singlepose_lightning_3.tflite")
    interpreter.allocate_tensors()

    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Reshape image to 192 x 192 x 3
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
        input_image = tf.cast(img, dtype=tf.float32)

        # Setup input and output
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()

        # Make predictions
        interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]["index"])

        # Rendering
        kp_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_kp(frame, keypoints_with_scores, 0.4)

        landmarks_list = keypoints_with_scores[0]

        # Check if there are landmarks
        if landmarks_list.shape[0] > 0:
            min_y = np.min(landmarks_list[:, 1])
            max_y = np.max(landmarks_list[:, 1])

            if np.isnan(min_y) or np.isnan(max_y):
                continue

            # Adjust the fall detection threshold here
            fall_threshold = 0.45
            
            print(max_y - min_y)
            if max_y - min_y > fall_threshold:
                # If fall is detected, add text to the frame
                cv2.putText(frame, "Fall detected", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        cv2.imshow("MoveNet Lightning", frame)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

### Testing

In [ ]:
video_path = "/Users/katarinajoanne/University/UGM/2 - Second Year/Fourth Semester/Computer Vision and Image Analysis/Assignment-3/sample.gif"
detect_falls(video_path)

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.3476678
0.32713383
0.35195696
0.39195523
0.39147675
0.31419393
0.2906052
0.280688
0.23341006
0.41338545
0.31842554
0.28146338
0.3685593
0.28407234
0.25404078
0.2394651
0.1978324
0.3092438
0.28858647
0.2976872
0.30154663
0.24915779
0.2479536
0.31968272
0.4895052
0.4614438
0.33695158
0.27919972
0.08504486
0.65303916
0.837806
0.2840091
0.34011954
0.35814255
0.3426305
0.29172313
0.51587576
0.5143788
0.42385086
0.6503502
0.56938666
0.17714322
0.74929833
0.758648
0.016179372
0.69269115
0.58629256
0.04363575
0.87239563
0.5888247


In [ ]:
video_path = "/Users/katarinajoanne/University/UGM/2 - Second Year/Fourth Semester/Computer Vision and Image Analysis/Assignment-3/sample_fall.gif"
detect_falls(video_path)

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.15849406
0.15275475
0.28265154
0.21493101
0.1924288
0.12905645
0.12905645
0.20393464
0.21435338
0.20720987
0.22378325
0.22412595
0.22412595
0.21945733
0.25663456
0.23805955
0.064587444
0.13187283
0.17948335
0.42659009
0.6676614
0.26745445
0.28531328
0.27921888
0.27921888
0.507524
0.25592315
0.2443895
0.4274596
0.4274596
